# Timing of oab
## Run these once, at the start of the notebook

In [ ]:
import logging
from time import perf_counter

import numpy as np
import pandas as pd
from rich import print

import oab

In [2]:
# make results list
results_table = []

## Personalize the following cell

In [3]:
# personalize this for running the notebook in different ways
algo_name = "RF"

how_many_images = 3  # how many images each combination of the above?

## Run these after each "personalization"
To get results on multiple datasets, algorithms, classes

### Timed cells

In [ ]:
datasets = ["mnist", "fashion", "emnist"]

for dataset_name in datasets:
    (X_train, Y_train), (X_test, Y_test), (X_tree, Y_tree) = oab.get_data(dataset_name)
    my_dom = oab.Domain(dataset_name, algo_name)
    index = {}
    points = {}
    for my_class in range(10):
        index[my_class] = np.where(Y_test == my_class)[0]
    my_dom.load()

    start = perf_counter()

    for my_class in my_dom.classes:
        for i, my_index in enumerate(index[my_class]):
            testpoint = oab.TestPoint(X_test[my_index], my_dom)
            logging.info(f"start explanation of index {my_index}")
            exp = oab.Explainer(testpoint, howmany=5)

            print(f"# factuals made: {len(exp.factuals)}")
            # counterfactuals part
            if exp.counterfactuals:
                print(f"how many counterfactuals made: {len(exp.counterfactuals)}")
            else:
                print("how many counterfactuals made: 0")

            if i >= how_many_images:
                # how many points to do per class
                break
        end = perf_counter()

        # compute time
        execution_time = end - start
        # append into results_table
        results_table.append(
            {
                "Dataset": dataset_name,
                "Algo": algo_name,
                "Class": my_class,
                "Time": execution_time,
            }
        )

## Run this to show the results table

In [ ]:
results_dataframe = pd.DataFrame.from_records(results_table)
results_dataframe